# Learning from Kaggle 1
## Melbourne Housing

In [2]:
import pandas as pd

melbourne_file_path = 'melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [3]:
melbourne_data = melbourne_data.dropna(axis=0)

### Selecting The Predicition Target

In [4]:
y = melbourne_data.Price

### Choosing "Features"
considering features(columns) for your prediction target

In [5]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']

In [6]:
X = melbourne_data[melbourne_features]

In [7]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,145.526350


In [8]:
X.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,144.9954


In [10]:
!pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/f1/7d/2e562207176a5dcdad513085670674bb11ffaf37e1393eacb6d7fb502481/scikit_learn-1.3.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/81/d7/d2537d51efb692d0c411e64267ba349e7668d40f5bc73cefe78ccd650dcd/scipy-1.11.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\soyou\anaconda3\envs\myenv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\soyou\anaconda3\envs\myenv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\soyou\anaconda3\envs\myenv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\soyou\anaconda3\envs\myenv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\soyou\anaconda3\envs\myenv\Lib\http\client.py", line 466, in read
    s = self.fp.read(amt)
        ^^^^^^^^^^^^^^^^^
  File "C:\Users\soyou\anaconda3\envs\myenv\Lib\socket.py", line 706, in read

   ------- -------------------------------- 7.8/44.1 MB 144.6 kB/s eta 0:04:12
   ------- -------------------------------- 7.8/44.1 MB 145.4 kB/s eta 0:04:10
   ------- -------------------------------- 7.8/44.1 MB 145.4 kB/s eta 0:04:10
   ------- -------------------------------- 7.9/44.1 MB 147.2 kB/s eta 0:04:06
   ------- -------------------------------- 7.9/44.1 MB 148.1 kB/s eta 0:04:05
   ------- -------------------------------- 7.9/44.1 MB 148.5 kB/s eta 0:04:04
   ------- -------------------------------- 8.0/44.1 MB 149.6 kB/s eta 0:04:02
   ------- -------------------------------- 8.0/44.1 MB 149.6 kB/s eta 0:04:02
   ------- -------------------------------- 8.0/44.1 MB 149.6 kB/s eta 0:04:02
   ------- -------------------------------- 8.0/44.1 MB 150.2 kB/s eta 0:04:01
   ------- -------------------------------- 8.0/44.1 MB 150.2 kB/s eta 0:04:01
   ------- -------------------------------- 8.0/44.1 MB 150.8 kB/s eta 0:04:00
   ------- -------------------------------- 8.1/44.1

## Building the Model

use the scikit-learn library. This library is written as sklearn.

In [12]:
from sklearn.tree import DecisionTreeRegressor

#Define model. Specify a number for random_state to ensure same results each run
melbourne_model = DecisionTreeRegressor(random_state=1)

#Fit model
melbourne_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

make predictions for the first few rows of the training data to see how the predict function works.

In [13]:
print("Making predictions for the following 5 houses:")
print(X.head())
print(melbourne_model.predict(X.head()))

Making predictions for the following 5 houses:
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
4      4       1.0     120.0   -37.8072    144.9941
6      3       2.0     245.0   -37.8024    144.9993
7      2       1.0     256.0   -37.8060    144.9954
[1035000. 1465000. 1600000. 1876000. 1636000.]


## Model Validation
calculate Mean Absolute Error(MAE)

error=actual−predicted


In [14]:
from sklearn.metrics import mean_absolute_error

predicted_home_prices = melbourne_model.predict(X)
mean_absolute_error(y, predicted_home_prices)

1115.7467183128902

The validation above is not good, because I used the same data to train and validate. For more accuracy, I need to split the data into two: one for train and one for validate to calculate mean_absolute_error(MAE)
To do this, train_test_split function from scikit-learn library

In [15]:
from sklearn.model_selection import train_test_split

#split data into training and validation data, for both features(X) and target(y)
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

#define model
melbourne_model = DecisionTreeRegressor()
#Fit model
melbourne_model.fit(train_X, train_y)

#get predicted prices on validation data
val_prediction = melbourne_model.predict(val_X)
print(mean_absolute_error(val_y, val_prediction))

272368.3514095115


## Overfitting and Underfitting
Overfitting: too many leaves. 
Underfitting: too few leaves.
Need to find the least Mean-Absolute-Error
Use a utility function to help compare MAE scores from different values for max_leaf_nodes

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [17]:
#compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d \t\t Mean Absolute Error: %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5 		 Mean Absolute Error: 385696
Max leaf nodes: 50 		 Mean Absolute Error: 279794
Max leaf nodes: 500 		 Mean Absolute Error: 261718
Max leaf nodes: 5000 		 Mean Absolute Error: 271320


In [20]:
best_tree_size = 500

## Fit Model Using All Data
Use the best tree size for more accurate results

In [21]:
final_model = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state = 0)
final_model.fit(X,y)

DecisionTreeRegressor(max_leaf_nodes=500, random_state=0)